Import Libraries

In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import os
import copy

# our modules
from train import *
from models import *
from datasets import *

Load Data

In [2]:
class args:
    def __init__(self):
        self.dataset="mnist"
        self.batch_size=64
        self.test_batch_size=1000
        self.device='cpu'
        self.permuted=True
        self.model="digits"
args = args()
train_loader, val_loader, test_loader, perm_train_loader, perm_val_loader, perm_test_loader = load_dataset(args)

train on permuted mnist

In [3]:
train_simple(perm_train_loader,perm_val_loader,args)

cpu
Train Epoch: 0 [0/50000 (0%)] train loss: 2.316, val loss: 2.398, val acc: 0.120 (best val acc: 0.000)
Train Epoch: 0 [12800/50000 (26%)] train loss: 0.218, val loss: 0.225, val acc: 0.935 (best val acc: 0.000)
Train Epoch: 0 [25600/50000 (51%)] train loss: 0.221, val loss: 0.207, val acc: 0.937 (best val acc: 0.000)
Train Epoch: 0 [38400/50000 (77%)] train loss: 0.123, val loss: 0.162, val acc: 0.948 (best val acc: 0.000)
In epoch 0, train loss: 0.200, val loss: 0.130, val acc: 0.961 (best val acc: 0.961))
Train Epoch: 1 [0/50000 (0%)] train loss: 0.029, val loss: 0.128, val acc: 0.962 (best val acc: 0.961)
Train Epoch: 1 [12800/50000 (26%)] train loss: 0.021, val loss: 0.113, val acc: 0.968 (best val acc: 0.961)
Train Epoch: 1 [25600/50000 (51%)] train loss: 0.035, val loss: 0.107, val acc: 0.968 (best val acc: 0.961)
Train Epoch: 1 [38400/50000 (77%)] train loss: 0.143, val loss: 0.117, val acc: 0.965 (best val acc: 0.961)
In epoch 1, train loss: 0.008, val loss: 0.097, val acc:

Load normal mnist

In [3]:
model = DigitClassifier()
checkpoint = torch.load("mnist_normal.pth")
state_dict = checkpoint['model_state_dict']
model.load_state_dict(state_dict)
model.eval()

DigitClassifier(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

train on permuted after normal

In [4]:
train_simple(perm_train_loader,perm_val_loader,args,model)

cpu
Train Epoch: 0 [0/50000 (0%)] train loss: 4.032, val loss: 2.240, val acc: 0.182 (best val acc: 0.000)
Train Epoch: 0 [19200/50000 (38%)] train loss: 0.075, val loss: 0.202, val acc: 0.938 (best val acc: 0.000)
Train Epoch: 0 [38400/50000 (77%)] train loss: 0.103, val loss: 0.159, val acc: 0.950 (best val acc: 0.000)
In epoch 0, train loss: 0.198, val loss: 0.143, val acc: 0.957 (best val acc: 0.957))
Train Epoch: 1 [0/50000 (0%)] train loss: 0.108, val loss: 0.132, val acc: 0.959 (best val acc: 0.957)
Train Epoch: 1 [19200/50000 (38%)] train loss: 0.179, val loss: 0.114, val acc: 0.965 (best val acc: 0.957)
Train Epoch: 1 [38400/50000 (77%)] train loss: 0.137, val loss: 0.099, val acc: 0.970 (best val acc: 0.957)
In epoch 1, train loss: 0.112, val loss: 0.109, val acc: 0.966 (best val acc: 0.966))


check forgetting

In [6]:
model = DigitClassifier()
checkpoint = torch.load("mnist_normal_perm.pth")
state_dict = checkpoint['model_state_dict']
model.load_state_dict(state_dict)
model.eval()
validate(model,perm_val_loader,args)

(0.9661, 0.10899652911126613)

In [ ]:
def perturb_params(model,amnt):
    with torch.no_grad():
        for idx,p in enumerate(model.parameters()):
            pert = torch.rand(p.size())
            p += (pert/torch.norm(pert)*amnt)


In [ ]:
losses = []
amnts = []
amnt = 0
for i in range(20):
    m = copy.deepcopy(model)
    perturb_params(m,amnt)

    losses.append(validate(m,val_loader,args))
    amnts.append(amnt)
    amnt += 0.025

In [ ]:
print(losses)

In [ ]:
ls = [l[1] for l in losses]
accs = [a[0] for a in losses]

ax1 = plt.subplot()
l1, = ax1.plot(amnts,ls)
ax2 = ax1.twinx()
l2, = ax2.plot(amnts,accs,color="orange")
# plt.scatter(amnts,ls)
# plt.scatter(amnts,accs)